In [12]:
from django.http.response import Http404, HttpResponseRedirect
from django.urls import reverse
from django.shortcuts import render, get_list_or_404, get_object_or_404
from django.views.decorators.clickjacking import xframe_options_sameorigin
from django.http import HttpResponse

import os
import sys
import django
from account.models import User

from django.utils import timezone
import datetime
import json
import random
import string

import hashlib
import base64

RuntimeError: Model class account.models.User doesn't declare an explicit app_label and isn't in an application in INSTALLED_APPS.

In [ ]:
def hash_and_compare(cracked_hash):
    cracked_chain = cracked_hash.split('$')
    # cracked_chain_digest = cracked_chain[0]
    cracked_chain_rounds = int(cracked_chain[1])
    cracked_chain_salt = cracked_chain[2].encode('utf-8')
    cracked_chain_hash_data = cracked_chain[3].split(':')
    cracked_chain_hash = base64.b64decode(cracked_chain_hash_data[0].encode('utf-8'))
    cracked_chain_data = cracked_chain_hash_data[1]

    hash_value = hashlib.pbkdf2_hmac('sha256', cracked_chain_data.encode('utf-8'), cracked_chain_salt, cracked_chain_rounds)
    hash_str = base64.b64encode(hash_value).decode('utf-8')
    salt_str = base64.b64encode(cracked_chain_salt).decode('utf-8')

    print(f'Computed: Hash: {hash_str} Salt: {salt_str}\nCracked: Hash: {cracked_chain_hash_data[0]} Salt: {cracked_chain[2]}\n')

hash_and_compare('pbkdf2_sha256$10000$005OtPxTXhPq$K/2GplWPJsBVj+qbgdKW8YEteQyUkIiquT5MaOhPo4Y=:harry')
hash_and_compare('pbkdf2_sha256$10000$00Qhibr5Mbeg$l9grYueDrl3qN3NA7e9j5PodgV1XkGTz0Z6ajhF99AY=:radio')
hash_and_compare('pbkdf2_sha256$10000$00h7h0g1ZKE1$YEobSm/y+cFg/VXhU4gGYJ6eOkZ68jhJ5axDu68Dack=:momo')
hash_and_compare('pbkdf2_sha256$10000$01JMkfGk1RXh$vD+GGZshw5kExtZOpl5+Lht3xECULdbNVOesoTicxto=:fred')
hash_and_compare('pbkdf2_sha256$10000$01vkw1viCg4J$2hjlbq10Jh/Su3yqjKfYCnCSt1WlKcKJtsqDET618M0=:get')
hash_and_compare('pbkdf2_sha256$10000$01wayF5JLVSZ$2/9COWqb6SZG/raqabtU8fNBzkrt2puN7SaKw0U7jBs=:987456321')


In [11]:
django.contrib.auth.hashers.PBKDF2PasswordHasher(salt='adf',iterations=1,salt_entropy=128)

TypeError: PBKDF2PasswordHasher() takes no arguments

In [7]:

hashlib.pbkdf2_hmac('sha256', '...'.encode('utf-8'), '...'.encode('utf-8'), 123456).hex()

'00d705e3bc857f46c6f73088302f7806e2c80e0a8755e3bef8ea01e98d79774c'

In [ ]:

os.chdir('/home/wxy/prj/Tostada')
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
from tool.models import AuthorizationLevel

code

In [ ]:
(User.objects.get()).__class__.__name__